# Project ideas

In [26]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
import time
# !pip install openpyxl
# import openpyxl 
from datetime import datetime
from functools import reduce

TASK ONE: Import file from given url

TASK TWO: Select rows
- Import file to pandas df
- Select only memes (category=="Meme")
- Drop duplicate rows
- Drop column Category

In [63]:
df = pd.read_json("kym.json")
df = df[df.category=="Meme"].reset_index(drop=True)
df=df.drop_duplicates(subset='title', keep='first')
df = df.drop("category",axis=1)
df = df.reset_index(drop=True)
df.shape

(12654, 15)

In [47]:
df.index

RangeIndex(start=0, stop=12654, step=1)

TASK THREE: skipped currently

TASK FOUR: Emptyness check - if loaded file (which is cleaned: selected memes and deleted duplicate rows) is empty then do not continue with following tasks, go to end

TASK FIVE: Select features
- From meta: ImageWidth, ImageHeight, SocialMediaDescription
- From details all: Status, Origin, Year, Type
- From content: AboutText, AboutLinks, AboutImages, OriginText, OriginLinks, OriginImages,
                       SpreadText, SpreadLinks', SpreadImages, NotExamplesText, NotExamplesLinks, NotExamplesImages,
                       SearchIntText, SearchIntLinks, SearchIntImages, ExtRefText, ExtRefLinks
- From main table all: Title, URL, TimeUpdate, Image, TimeAdded, Tags, References, SearchKeywords, Parent (url)

In [48]:
# select only wanted data features, flatten the data
# META:
def _meta_to_df(df):
    meta_data = pd.DataFrame(df['meta'].values.tolist())
    meta_data = meta_data[['og:image:width', 'og:image:height', 'og:description']]
    meta_data = meta_data.rename(
        {'og:image:width': 'ImageWidth', 'og:image:height': 'ImageHeight',
         'og:description': 'SocialMediaDescription'},
        axis=1)
    return meta_data

# DETAILS:
def _details_to_df(df):
    details = pd.DataFrame(df['details'].values.tolist())
    details = details[['status','origin','year','type']]
    details = details.rename(
        {'status':'Status','origin':'Origin','year':'Year','type':'Type'}
    )
    return details

# CONTENT SUBTABLES:
def norm_cont(df,col,subcols):
    subtable = pd.json_normalize(df[f"{col}"])
    subtable = subtable[subcols]
    return subtable

# CONTENT:
def _content_to_df(df):
    content = pd.DataFrame(df['content'].values.tolist())
    sub_columns = ["text","links","images"]
    about = norm_cont(content,"about",sub_columns)
    origin = norm_cont(content,"origin",sub_columns)
    spread = norm_cont(content,"spread",sub_columns)
    notable_examples = norm_cont(content,"notable examples",sub_columns)
    search_interest = norm_cont(content,"search interest",sub_columns)
    external_references = norm_cont(content,"external references",sub_columns[:-1]) # images tulpa external_ref tabelis ei ole
    content = pd.concat([about, origin, spread, notable_examples, search_interest, external_references],axis=1)
    # kuna tulpade nimed on korduvad, siis tuleks defineerida neid ümber hoopis nii
    content.columns = ['AboutText','AboutLinks', 'AboutImages', 'OriginText', 'OriginLinks', 'OriginImages',
                       'SpreadText', 'SpreadLinks', 'SpreadImages', 'NotExamplesText', 'NotExamplesLinks', 'NotExamplesImages',
                       'SearchIntText', 'SearchIntLinks', 'SearchIntImages', 'ExtRefText', 'ExtRefLinks']
    return content

In [64]:
# MAIN FN FOR SELECTING FEATURES:
main = df[['title','url','last_update_source','template_image_url','added','tags','additional_references','search_keywords','parent']]
print('main: ',main.shape)
meta = _meta_to_df(df)
print('meta: ',meta.shape)
details = _details_to_df(df)
print('details: ',details.shape)
content = _content_to_df(df)
print('content: ',content.shape)

main:  (12654, 9)
meta:  (12654, 3)
details:  (12654, 4)
content:  (12654, 17)


In [66]:
# CONCATENATION
df = pd.concat([main,meta,details,content],axis=1)
print('concat: ',df.shape)
df.columns = ['Title','URL','TimeUpdated','Image','TimeAdded','Tags','References','SearchKeywords','Parent', 
              'ImageWidth', 'ImageHeight', 'SocialMediaDescription',
              'Status', 'Origin','Year', 'Type','AboutText', 'AboutLinks', 
              'AboutImages', 'OriginText', 'OriginLinks', 'OriginImages',
              'SpreadText', 'SpreadLinks', 'SpreadImages', 'NotExamplesText',
              'NotExamplesLinks', 'NotExamplesImages', 'SearchIntText',
              'SearchIntLinks', 'SearchIntImages', 'ExtRefText', 'ExtRefLinks']


concat:  (12654, 26)
concat2:  (12654, 33)


In [70]:
df.isnull().sum()

Title                         0
URL                           0
TimeUpdated                   0
Image                         0
TimeAdded                   156
Tags                          0
References                    0
SearchKeywords             6992
Parent                     7325
ImageWidth                    0
ImageHeight                   0
SocialMediaDescription       16
Status                        0
Origin                        0
Year                       1094
Type                       9885
AboutText                  5047
AboutLinks                 6282
AboutImages               12467
OriginText                 5531
OriginLinks                8046
OriginImages               9244
SpreadText                 6153
SpreadLinks                6931
SpreadImages               9099
NotExamplesText           12251
NotExamplesLinks          12432
NotExamplesImages         11453
SearchIntText             11465
SearchIntLinks            12572
SearchIntImages           12646
ExtRefTe

TASK SIX: The time fields (TimeAdded, TimeUpdated) are formatted to ...

In [170]:
df['TimeAdded'] = df['TimeAdded'].fillna(-2208988800)

In [172]:
# convert epoch time to readable date time
def format_time(col_name):
   formatted_date_time=pd.to_datetime(col_name.apply(datetime.fromtimestamp))
   return formatted_date_time

# Column to be formatted from miliseconds
df['TimeUpdated'] = format_time(df['TimeUpdated'])
df['TimeAdded'] = format_time(df['TimeAdded'])

In [174]:
df.sample(5)

,Title,URL,TimeUpdated,Image,TimeAdded,Tags,References,SearchKeywords,Parent,ImageWidth,...,SpreadLinks,SpreadImages,NotExamplesText,NotExamplesLinks,NotExamplesImages,SearchIntText,SearchIntLinks,SearchIntImages,ExtRefText,ExtRefLinks
4269,You Make My Kokoro Go Doki Doki,https://knowyourmeme.com/memes/you-make-my-kok...,2019-11-19 23:32:32,https://i.kym-cdn.com/entries/icons/original/0...,2016-01-06 01:59:25,"[japanese, you make my heart go, kokoro, doki ...",{'Urban Dictionary': 'https://www.urbandiction...,"[kokoro doki doki, kokoro go doki doki]",https://knowyourmeme.com/memes/cultures/anime-...,600,...,"[[Urban Dictionary, https://knowyourmeme.com/m...",[https://i.kym-cdn.com/photos/images/newsfeed/...,NaN,NaN,NaN,NaN,NaN,NaN,[[1] Yahoo! Answers – You Make My Kokoro Go Do...,"[[You Make My Kokoro Go Doki Doki?, https://an..."
3658,Customer Support Chat Trolling,https://knowyourmeme.com/memes/customer-suppor...,2019-11-19 23:32:43,https://i.kym-cdn.com/entries/icons/original/0...,2016-07-20 23:47:18,"[prank, troll, trolling, customer service, web...",{},NaN,https://knowyourmeme.com/memes/cultures/pranking,600,...,"[[gamer, https://knowyourmeme.com/memes/cultur...","[http://i.imgur.com/Nt9zFAz.jpg, http://i.imgu...",NaN,NaN,NaN,[Not available.],NaN,NaN,[[1] Reddit – /b/ trolls Apple Customer Suppor...,"[[/b/ trolls Apple Customer Support chat, http..."
10609,History Through Facebook,https://knowyourmeme.com/memes/history-through...,2018-07-18 00:07:52,https://i.kym-cdn.com/entries/icons/original/0...,2010-09-24 21:51:51,"[facebook, parody, history]",{},NaN,https://knowyourmeme.com/memes/subcultures/his...,600,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6898,Golden Eagle Snatches Kid,https://knowyourmeme.com/memes/golden-eagle-sn...,2019-12-18 00:08:45,https://i.kym-cdn.com/entries/icons/original/0...,2012-12-20 20:22:56,"[viral video, fake, hoax, entry of december 20...",{},[golden eagle snatches],NaN,600,...,"[[Redditor, https://knowyourmeme.com/memes/sit...",[https://i.kym-cdn.com/photos/images/newsfeed/...,NaN,NaN,[https://i.kym-cdn.com/photos/images/newsfeed/...,NaN,NaN,NaN,[[1] Reddit – Golden Eagle Snatches Kid in Mon...,"[[Golden Eagle Snatches Kid in Montreal, https..."
2401,Spambot's Cold Bed,https://knowyourmeme.com/memes/spambots-cold-bed,2017-06-14 15:52:42,https://i.kym-cdn.com/entries/icons/original/0...,1900-01-01 01:39:00,"[bed, spambot, spam, virus, e3, microsoft, nin...",{},NaN,NaN,600,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


TASK SEVEN: Format fields

TASK EIGHT: remove sensitive/impropriate data

In [175]:
df.Tags

0        [image macros, comment, comments, approval, im...
1         [ascii, animated gif, rofl, acronym, initialism]
2        [myspace, 4chan, parody, know, bitches, dont, ...
3        [youtube, rant, britney spears, response, frea...
4        [image macro, owl, orly, o rly, negative, some...
                               ...                        
12649    [bird suit, muppets grammys, cee lo, tumblr, v...
12650    [advice dog, hypocrite, liberal, janeane garof...
12651    [game, creepypasta, fanart, nightmare, fangame...
12652    [donald trump, ted cruz, 2016 republican natio...
12653    [ytpmv, final fantasy, chocobo racing, japan, ...
Name: Tags, Length: 12654, dtype: object

In [203]:
df.Tags.str.contains('nsfw').unique()

array([nan])

In [210]:
df.shape

(12654, 33)

In [212]:
# create intermediate table nsfw
nsfw = df[['URL','Tags']]
nsfw["Tags"] = nsfw["Tags"].astype('str')
t1 = nsfw.apply(lambda row: row["Tags"].split('[')[-1], axis=1) #.split(']')[0].split(',')
nsfw["t1"]=t1
t2 = nsfw.apply(lambda row: row["t1"].split(']')[0], axis=1) #.split(',')
nsfw["t2"]=t2
SingleTag = nsfw.apply(lambda row: row["t2"].split(','), axis=1).explode()
nsfw = nsfw.join(pd.DataFrame(SingleTag,columns=["Tag"]))
nsfw = nsfw.drop(["Tags","t1","t2"],axis=1)
# Filter nsfw from Tags
nsfw = nsfw.Tag.str.contains('nsfw')
print(nsfw.shape)
nsfw = nsfw.reset_index()
print(nsfw.shape)
#nsfw.sample(3)
nsfw = nsfw.drop_duplicates()
nsfw = nsfw.reset_index(index=nsfw.index)
print(nsfw.shape)
nsfw.sample(3)



/var/folders/hk/kbn1q_sj4jz4lh0m7x05vff80000gn/T/ipykernel_22216/1774149021.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nsfw["Tags"] = nsfw["Tags"].astype('str')
/var/folders/hk/kbn1q_sj4jz4lh0m7x05vff80000gn/T/ipykernel_22216/1774149021.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nsfw["t1"]=t1
/var/folders/hk/kbn1q_sj4jz4lh0m7x05vff80000gn/T/ipykernel_22216/1774149021.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer

(74997,)
(74997, 2)


TypeError: reset_index() got an unexpected keyword argument 'index'

TASK NINE: Create SQL script

TASK TEN: Insert to DB